# RandAugment Tutorial

- 실습조교: 배진수(wlstn215@korea.ac.kr), 안시후(sihuahn@korea.ac.kr), 김현지(99ktxx@korea.ac.kr)

## Colab gpu 연결

### 런타임 -> 런타임유형 변경 -> 하드웨어 가속기 -> GPU

In [ ]:
import torch
torch.cuda.is_available()

## 0.모듈 불러오기

In [ ]:
''' github+colab 교육생분들 '''
# !git clone https://github.com/bogus215/LG-EDUCATION3.git

import matplotlib.pyplot as plt
import numpy as np
import PIL
import pandas as pd
from PIL import Image
from torchvision import datasets
from augmentation import *

## 1. CIFAR 10 Dataset 다운로드 및 확인

In [ ]:
# 학습 및 테스팅 데이터 다운로드 및 저장

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True)

### 탐색적 데이터 분석

In [ ]:
# 데이터 구조 확인하기
print("CIFAR10 클래스 종류 리스트: ",train_dataset.classes)
print("CIFAR10 클래스 이름 및 인덱스: ",train_dataset.class_to_idx)

print("학습 데이터 Feature SHAPE: ",train_dataset.data.shape)
print("테스팅 데이터 Feature SHAPE: ",test_dataset.data.shape)

print("학습 데이터 Target 개수: ",len(train_dataset.targets))
print("테스팅 데이터 Target 개수: ",len(test_dataset.targets))

In [ ]:
# 학습 데이터 클래스 비율 확인하기
print(pd.value_counts(train_dataset.targets))

In [ ]:
# 테스팅 데이터 클래스 비율 확인하기
print(pd.value_counts(test_dataset.targets))

In [ ]:
# CIFAR10 이미지 데이터 그려보기

fig = plt.figure(figsize=(20, 10))
for cls, i in zip(train_dataset.class_to_idx.keys(), train_dataset.class_to_idx.values()):
    index_list = np.where(np.array(train_dataset.targets)==i)[0]

    index = np.random.RandomState(0).choice(index_list, 1)

    img = train_dataset.data[index][0] # width X height x channel
    
    fig.add_subplot(2, 5, i+1)
    plt.imshow(img)
    plt.title(f'{cls} - {i}')

plt.show()

## 2. RandAugment 적용하기

### 비행기 이미지 사진에 적용해보기

In [ ]:
# RandAugment에서 사용할 augmentation 리스트 생성
# Augmentation별 강도 조절 및 환경과 관련된 하이퍼파라미터 설정 필요
augment_list = [
                (AutoContrast, None, None), 
                (Equalize, None, None),
                (Invert, None, None),
                (Rotate, 30, None),
                (Posterize, 4, 0),
                (Solarize, 256, None),
                (Color, 1.8, 0.1),
                (Contrast, 1.8, 0.1),
                (Brightness, 1.8, 0.1),
                (Sharpness, 1.8, 0.1),
                (ShearX, 0.3, None),
                (ShearY, 0.3, None),
                (TranslateXConst, 100, None),
                (TranslateYConst, 100, None)
                ]

In [ ]:
# RandAugment 클래스 만들기
class RandAugment(object):
    def __init__(self, n, m, augment_list):
        
        self.n = int(n) # Augmentation 중복 적용 횟수
        self.m = m # Augmentation 강도 조절
        self.augment_pool = augment_list # Augmentation 리스트

    def __call__(self, img):
        ops = random.choices(self.augment_pool, k=self.n) # Augmentation 리스트에서 랜덤 선택
        for op, max_v, bias in ops: # 선택된 증강 기법들을 중복하여 적용
            img = op(img, v=self.m, max_v=max_v, bias=bias)
        return img

In [ ]:
# 고화질 비행기 이미지 사진에 RandAugment 적용해보기
image = Image.open('./data/air_plane.jpg').convert("RGB")
image

In [ ]:
# RandAugment 적용하기
n, m = 3, 5
ra = RandAugment(n, m, augment_list=augment_list)
transform_image = ra(image)

In [ ]:
# RandAugment 적용된 이미지 사진 시각화 하기 (Augment 적용 전 사진과 비교)
fig = plt.figure(figsize=(20, 10))

fig.add_subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original')

fig.add_subplot(1, 2, 2)
plt.imshow(transform_image)
plt.title(f'RandAugment n={n}, m={m}')

plt.show()

### CIFAR 10 데이터에 적용하기

In [ ]:
# RandAugment에서 사용할 augmentation 리스트 생성
# Augmentation별 강도 조절 및 환경과 관련된 하이퍼파라미터 설정 필요
cifar10_aug_list = [
                    (AutoContrast, None, None),
                    (Equalize, None, None),
                    (Invert, None, None),
                    (Rotate, 30, None),
                    (Posterize, 4, 0),
                    (Solarize, 256, None),
                    (Color, 1.8, 0.1),
                    (Contrast, 1.8, 0.1),
                    (Brightness, 1.8, 0.1),
                    (Sharpness, 1.8, 0.1),
                    (ShearX, 0.3, None),
                    (ShearY, 0.3, None),
                    (TranslateXConst, 32 // 8, None), # CIFAR10 이미지 크기: 32
                    (TranslateYConst, 32 // 8, None),
                    (CutoutConst, 32 // 8, None),
                    ]

In [ ]:
# RandAugment 클래스 만들기 + CutOut 추가
class RandAugmentCIFAR(object):
    def __init__(self, n, m, augment_list):

        self.n = int(n)
        self.m = m
        self.augment_pool = augment_list

    def __call__(self, img):
        ops = random.choices(self.augment_pool, k=self.n)
        for op, max_v, bias in ops:
            img = op(img, v=self.m, max_v=max_v, bias=bias)
        return img

In [ ]:
ra_cifar = RandAugmentCIFAR(5, 5, augment_list=cifar10_aug_list)

In [ ]:
fig = plt.figure(figsize=(10,50))
for cls, i in zip(train_dataset.class_to_idx.keys(), train_dataset.class_to_idx.values()):
    index_list = np.where(np.array(train_dataset.targets)==i)[0]
    index = np.random.RandomState(0).choice(index_list, 1)

    img = train_dataset.data[index][0]
    img = Image.fromarray(img)
    transform_image = ra_cifar(img)
    
    fig.add_subplot(10, 2, 2*i+1)
    plt.imshow(img)
    plt.title(f'{cls} - {i}')
    
    fig.add_subplot(10, 2, 2*i+2)
    plt.imshow(transform_image)
    plt.title(f'{cls} - {i} RandAug')
plt.show()